# Imports and pip install

In [1]:
#pip install
!pip install firebase
!pip install requests beautifulsoup4

#imports
import requests
from IPython.display import HTML, display
import ipywidgets as widgets
from IPython.display import clear_output
from datetime import datetime
import IPython
import json
from firebase_admin import db
from google.colab import files
import firebase_admin
from firebase_admin import credentials
from collections import Counter
import matplotlib.pyplot as plt
import io
import base64
from textwrap import wrap
import numpy as np
from google.colab import drive
import re
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import random
from firebase import firebase

In [2]:
#connection to firebase and drive
drive.mount('/content/drive')
FBconn=firebase.FirebaseApplication('https://homework2-12bd2-default-rtdb.asia-southeast1.firebasedatabase.app/',None)

Mounted at /content/drive


# Here is the part we save the JSON file data we received for our team in firebase

In [3]:
# Path to the json file in your Google Drive
file_path = '/content/drive/My Drive/data.json'
with open(file_path, "r") as file:
    data = json.load(file)
#save the json file in firebase
result = FBconn.put('/', 'Data', data)
print('Data uploaded successfully!')

Data uploaded successfully!


# Here is the part we build an index of the significant words on the onShape glossary page

In [4]:
def fetch_page(url):
  response = requests.get(url)
  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup
  else:
    return None

In [5]:
def index_words(soup):
  index = {}
  words = re.findall(r'\w+', soup.get_text())
  for word in words:
    word = word.lower()
    if word in index:
      index[word] += 1
    else:
      index[word] = 1
  return index

In [6]:
def remove_stop_words(index):
  stop_words = {'a', 'an', 'the', 'and', 'or','in', 'on', 'at'}
  for stop_word in stop_words:
    if stop_word in index:
      del index[stop_word]
  return index

In [7]:
def apply_stemming(index):
  stemmer = PorterStemmer()
  stemmed_index = {}
  for word, count in index.items():
    stemmed_word = stemmer.stem(word)
    if stemmed_word in stemmed_index:
      stemmed_index[stemmed_word] += count
    else:
      stemmed_index[stemmed_word] = count
  return stemmed_index

In [8]:
def search(query, index):
  stemmer = PorterStemmer()
  query_words = re.findall(r'\w+', query.lower())
  results = {}
  for word in query_words:
    word = stemmer.stem(word)
    if word in index:
      results[word] = index[word]
  return results

In [9]:
def search_engine(url, query):
  soup = fetch_page(url)
  if soup is None:
    return None
  index = index_words(soup)
  index = remove_stop_words(index)
  index = apply_stemming(index)
  results = search(query, index)
  return results

In [10]:
def find_index(url):
  soup = fetch_page(url)
  if soup is None:
    return None
  index = index_words(soup)
  index = remove_stop_words(index)
  index = apply_stemming(index)
  return index

In [11]:
def find_top_n_elements(objects, n=10):
    if len(objects) <= n:
        return sorted(objects, key=lambda x: x['freq'], reverse=True)

    # Initialize an empty list to store the top n elements
    top_n = []

    # Iterate through the array of objects
    for obj in objects:
        if len(top_n) < n:
            top_n.append(obj)
            top_n.sort(key=lambda x: x['freq'], reverse=True)
        else:
            if obj['freq'] > top_n[-1]['freq']:  # Compare with the smallest frequency in top_n
                top_n[-1] = obj
                top_n.sort(key=lambda x: x['freq'], reverse=True)

    return top_n

In [12]:
#find the index for each word in the url , also find the most 10 words that have most frequent
def add_index_to_firebase(url):
    firebase_url = 'https://homework2-12bd2-default-rtdb.asia-southeast1.firebasedatabase.app/'
    FBconn = firebase.FirebaseApplication(firebase_url,None)
    result = find_index(url)
    index=[]
    for word,sum in result.items():
      data={
          'freq':sum,
          'term':word
      }
      index.append(data)
    top_10=find_top_n_elements(index)
    result = FBconn.put('/', 'Top_10_word', top_10)
    print(result)
    result = FBconn.put('/', 'Index', index)
    print(result)

add_index_to_firebase('https://cad.onshape.com/help/Content/Glossary/glossary.htm?tocpath=_____19')

[{'freq': 676, 'term': 'context'}, {'freq': 656, 'term': 'type'}, {'freq': 648, 'term': 'also'}, {'freq': 644, 'term': 'all'}, {'freq': 638, 'term': 'see'}, {'freq': 631, 'term': 'keyboard'}, {'freq': 631, 'term': 'shortcut'}, {'freq': 626, 'term': 'plan'}, {'freq': 524, 'term': 'part'}, {'freq': 522, 'term': 'to'}]
[{'freq': 3, 'term': 'glossari'}, {'freq': 1, 'term': 'skip'}, {'freq': 522, 'term': 'to'}, {'freq': 2, 'term': 'main'}, {'freq': 10, 'term': 'content'}, {'freq': 33, 'term': 'account'}, {'freq': 33, 'term': 'set'}, {'freq': 2, 'term': 'logout'}, {'freq': 1, 'term': 'placehold'}, {'freq': 16, 'term': 'filter'}, {'freq': 644, 'term': 'all'}, {'freq': 24, 'term': 'file'}, {'freq': 1, 'term': 'submit'}, {'freq': 13, 'term': 'search'}, {'freq': 676, 'term': 'context'}, {'freq': 86, 'term': 'option'}, {'freq': 164, 'term': 'document'}, {'freq': 220, 'term': 'sketch'}, {'freq': 524, 'term': 'part'}, {'freq': 369, 'term': 'studio'}, {'freq': 279, 'term': 'assembl'}, {'freq': 226, 

In [13]:
#find the index for 10 words selected
results = []
url = 'https://cad.onshape.com/help/Content/Glossary/glossary.htm?tocpath=_____19'

# Execute the search engine function for each word and store the results
query = ['point', 'keyboard', 'context', 'plan', 'type', 'draw', 'opens', 'allow', 'sketch', 'part']

for word in query:
    result = search_engine(url, word)
    results.append({
        'term': word,
        'freq': result
    })

index = []

for entry in results:
    data = {
        'term': entry['term'],
        'freq': entry['freq']
    }
    index.append(data)

result = FBconn.put('/', 'Selected_10_word', index)
print(result)

[{'freq': {'point': 74}, 'term': 'point'}, {'freq': {'keyboard': 631}, 'term': 'keyboard'}, {'freq': {'context': 676}, 'term': 'context'}, {'freq': {'plan': 626}, 'term': 'plan'}, {'freq': {'type': 656}, 'term': 'type'}, {'freq': {'draw': 226}, 'term': 'draw'}, {'freq': {'open': 61}, 'term': 'opens'}, {'freq': {'allow': 67}, 'term': 'allow'}, {'freq': {'sketch': 220}, 'term': 'sketch'}, {'freq': {'part': 524}, 'term': 'part'}]


#Here is the second part of the home work, building the system , and prepare the screens

**The upload file screen**


In [14]:
# Firebase URL
firebase_url = 'https://homework2-12bd2-default-rtdb.asia-southeast1.firebasedatabase.app/Data.json'

# Fetch data from Firebase
response = requests.get(firebase_url)
data = response.json()

**The most frequent activity page**

In [15]:
def create_most_frequent_activity_page():
    user_entries = []
    if 'data' in globals() and isinstance(data, list):
        for entry in data:
            if isinstance(entry, dict) and 'User' in entry and 'Description' in entry:
                user_entries.append(entry)

    # Count occurrences of each activity
    activity_counts = Counter(entry['Description'] for entry in user_entries)
    top_activities = activity_counts.most_common(5)
    activities = [activity for activity, _ in top_activities]
    counts = [count for _, count in top_activities]

    # Create a new figure
    plt.figure(figsize=(10, 6))

    # Plotting the bar chart
    plt.barh(activities, counts, color='skyblue')
    plt.title('Top 5 Most Frequent Activities', fontsize=15)
    plt.xlabel('Count', fontsize=12)
    plt.ylabel('Activities', fontsize=12)

    # Adding counts as text on the bars
    for i, count in enumerate(counts):
        plt.text(count, i, str(count), va='center', fontsize=10)

    plt.gca().invert_yaxis()  # Invert y-axis to display the most frequent activity at the top
    plt.tight_layout()  # Adjust layout to prevent overlapping

    # Save the plot to a bytes buffer
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_str = base64.b64encode(buf.getvalue()).decode('utf-8')

    # Close the matplotlib figure to free up memory
    plt.close()

    # Display the chart as an image in HTML
    html_content = f"""
    <html>
    <head>
        <title>Most Frequent Activities</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                background-color: #f0f0f0;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
                margin: 0;
            }}
            .container {{
                background-color: white;
                border-radius: 10px;
                padding: 20px;
                box-shadow: 0 0 10px rgba(0,0,0,0.1);
            }}
            h1 {{
                color: #333;
                text-align: center;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Most Frequent Activities</h1>
            <img src="data:image/png;base64,{img_str}"/>
        </div>
    </body>
    </html>
    """

    display(HTML(html_content))

    # Add a back button
    back_button = widgets.Button(description='Back to Main')
    back_button.on_click(show_main_page)
    display(back_button)

**The activity distribution over time page**

In [16]:
def create_activity_distribution_over_time_page():
    user_entries = []
    if 'data' in globals() and isinstance(data, list):
        for entry in data:
            if isinstance(entry, dict) and 'User' in entry and 'Time' in entry:
                user_entries.append(entry)

    # Extract timestamps
    timestamps = [datetime.strptime(entry['Time'], '%Y-%m-%d %H:%M:%S') for entry in user_entries]

    # Create the histogram
    plt.figure(figsize=(12, 6))
    plt.hist(timestamps, bins=50, color='skyblue', edgecolor='black')
    plt.title('Activity Distribution Over Time', fontsize=15)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Number of Activities', fontsize=12)
    plt.tick_params(axis='x', rotation=45, labelsize=8)
    plt.tick_params(axis='y', labelsize=8)

    plt.tight_layout()  # Adjust layout to prevent overlapping

    # Save the plot to a bytes buffer
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_str = base64.b64encode(buf.getvalue()).decode('utf-8')

    # Close the matplotlib figure to free up memory
    plt.close()

    # Display the chart as an image in HTML
    html_content = f"""
    <html>
    <head>
        <title>Activity Distribution Over Time</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                background-color: #f0f0f0;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
                margin: 0;
            }}
            .container {{
                background-color: white;
                border-radius: 10px;
                padding: 20px;
                box-shadow: 0 0 10px rgba(0,0,0,0.1);
            }}
            h1 {{
                color: #333;
                text-align: center;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <img src="data:image/png;base64,{img_str}"/>
        </div>
    </body>
    </html>
    """

    display(HTML(html_content))

    # Add a back button
    back_button = widgets.Button(description='Back to Main')
    back_button.on_click(show_main_page)
    display(back_button)

**The most frequent worker name page**

In [17]:
def create_on_most_frequent_worker_names_page():
    user_entries = []
    if 'data' in globals() and isinstance(data, list):
        for entry in data:
            if isinstance(entry, dict) and 'User' in entry:
                user_entries.append(entry)

    # Count occurrences of each name
    name_counts = Counter(entry['User'] for entry in user_entries)

    # Sort names by count in descending order
    sorted_names = sorted(name_counts.items(), key=lambda x: x[1], reverse=True)

    html_content = """
    <style>
        body { font-family: Arial, sans-serif; }
        table {
            border-collapse: collapse;
            width: 40%;
            margin: 10px auto;
            background-color: white;
        }
        th, td {
            border: 1px solid black;
            padding: 4px;
            text-align: left;
            background-color: white;
            color: black;
        }
        th {
            background-color: #800080;
            color: white;
        }
        h1, h2 {
            text-align: center;
            color: black;
        }
        .chart-container {
            text-align: center;
            margin: 10px 0;
        }
    </style>
    <h2>Most Frequent Worker Names</h2>
    <table>
        <tr>
            <th>Rank</th>
            <th>Name</th>
            <th>Count</th>
        </tr>
    """

    for i, (name, count) in enumerate(sorted_names, 1):
        html_content += f"<tr><td>{i}</td><td>{name}</td><td>{count}</td></tr>"

    html_content += "</table>"

    display(HTML(html_content))

    # Add a back button
    back_button = widgets.Button(description='Back to Main')
    back_button.on_click(show_main_page)
    display(back_button)


**Recent Activity Details Page**

In [18]:
def create_on_recent_activity_details_button_click_page():
    user_entries = []
    if 'data' in globals() and isinstance(data, list):
        for entry in data:
            if isinstance(entry, dict) and 'User' in entry:
                user_entries.append(entry)

    html_content = """
    <style>
        body { font-family: Arial, sans-serif; }
        table {
            border-collapse: collapse;
            width: 40%;
            margin: 10px auto;
            background-color: white;
        }
        th, td {
            border: 1px solid black;
            padding: 4px;
            text-align: left;
            background-color: white;
            color: black;
        }
        th {
            background-color: #800080;
            color: white;
        }
        h1, h2 {
            text-align: center;
            color: black;
        }
        .chart-container {
            text-align: center;
            margin: 10px 0;
        }
    </style>
    <h2>Recent Activity Details</h2>
    <table>
        <tr>
            <th>User</th>
            <th>Description</th>
            <th>Document</th>
            <th>Tab</th>
            <th>Time</th>
        </tr>
    """

    # Add rows to the details table (showing last 20 entries)
    for entry in user_entries[-20:]:
        html_content += f"""
        <tr>
            <td>{entry.get('User', 'N/A')}</td>
            <td>{entry.get('Description', 'N/A')}</td>
            <td>{entry.get('Document', 'N/A')}</td>
            <td>{entry.get('Tab', 'N/A')}</td>
            <td>{entry.get('Time', 'N/A')}</td>
        </tr>
        """

    html_content += """
    </table>
    """

    # Display the HTML content
    display(HTML(html_content))

    # Add a back button
    back_button = widgets.Button(description='Back to Main')
    back_button.on_click(show_main_page)
    display(back_button)

In [19]:
# @title Stop here Before Executin the next cell. { display-mode: "both" }
""" Please Pay attention before executing the next cell.
In order to run the code properly with the right logical functionality an admin sdk file must be downloaded and found at your drive(/content/drive/mydrive/ the name of the file.json).
You can download the file from the following link,https://console.firebase.google.com/u/0/project/homework2-12bd2/settings/serviceaccounts/adminsdk.
Press on "Generate new private key", and the sdk file should be downloaded to you device.
Simply drag and drop the file to your drive, P.S : specifaclly in האחסון שלי
After having the file in your drive copy its name and paste it in the next cell under the imports in this line     cred = credentials.Certificate("/content/drive/My Drive/homework2-12bd2-firebase-adminsdk-d8w1y-65ac871075.json")
where himework2-12bd2.... should now be your sdk file.
Your'e good to go now :)"""


' Please Pay attention before executing the next cell.\nIn order to run the code properly with the right logical functionality an admin sdk file must be downloaded and found at your drive(/content/drive/mydrive/ the name of the file.json).\nYou can download the file from the following link,https://console.firebase.google.com/u/0/project/homework2-12bd2/settings/serviceaccounts/adminsdk.\nPress on "Generate new private key", and the sdk file should be downloaded to you device.\nSimply drag and drop the file to your drive, P.S : specifaclly in האחסון שלי\nAfter having the file in your drive copy its name and paste it in the next cell under the imports in this line     cred = credentials.Certificate("/content/drive/My Drive/homework2-12bd2-firebase-adminsdk-d8w1y-65ac871075.json")\nwhere himework2-12bd2.... should now be your sdk file.\nYour\'e good to go now :)'

In [20]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt
import io
import base64
from firebase import firebase
import json
import firebase_admin
from firebase_admin import credentials, db
from collections import Counter
from datetime import datetime, timedelta

# Firebase initialization
if not firebase_admin._apps:
    cred = credentials.Certificate("/content/drive/My Drive/homework2-12bd2-firebase-adminsdk-d8w1y-6eb30c4efd.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://homework2-12bd2-default-rtdb.asia-southeast1.firebasedatabase.app/'
    })

FBconn = firebase.FirebaseApplication('https://homework2-12bd2-default-rtdb.asia-southeast1.firebasedatabase.app/', None)

# Global variable to store the data
data = []

# Function to fetch data from Firebase
def fetch_data_from_firebase():
    global data
    data = FBconn.get('/Data', None)

# Fetch data at the beginning
fetch_data_from_firebase()

# Function to create the upload page
def create_upload_page():
    html_content = """
    <html>
    <head>
        <title>Stylish JSON File Upload</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
                margin: 0;
            }
            .container {
                background-color: rgba(255, 255, 255, 0.9);
                border-radius: 10px;
                padding: 30px;
                box-shadow: 0 10px 20px rgba(0,0,0,0.19), 0 6px 6px rgba(0,0,0,0.23);
                text-align: center;
            }
            #drop_zone {
                border: 3px dashed #7e57c2;
                border-radius: 10px;
                width: 300px;
                height: 150px;
                padding: 20px;
                margin: 20px auto;
                display: flex;
                flex-direction: column;
                justify-content: center;
                align-items: center;
                cursor: pointer;
                transition: all 0.3s ease;
            }
            #drop_zone.highlight {
                background-color: #e8eaf6;
                border-color: #3f51b5;
            }
            #drop_zone.file-dropped {
                background-color: #c8e6c9;
                border-color: #4caf50;
            }
            #upload_btn {
                background-color: #4CAF50;
                border: none;
                color: white;
                padding: 15px 32px;
                text-align: center;
                text-decoration: none;
                display: inline-block;
                font-size: 16px;
                margin: 4px 2px;
                cursor: pointer;
                border-radius: 5px;
                transition: all 0.3s ease;
            }
            #upload_btn:hover {
                background-color: #45a049;
            }
            h1 {
                color: #333;
                margin-bottom: 20px;
            }
            #file_name {
                margin-top: 10px;
                font-weight: bold;
                color: #4caf50;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Upload JSON to Firebase</h1>
            <div id="drop_zone">
                <p id="drop_text">Drop JSON file here or click to select</p>
                <p id="file_name"></p>
            </div>
            <input type="file" id="file_input" style="display: none;" accept=".json">
            <button id="upload_btn" onclick="uploadFile()">Upload to Firebase</button>
        </div>

        <script>
            var dropZone = document.getElementById('drop_zone');
            var fileInput = document.getElementById('file_input');
            var dropText = document.getElementById('drop_text');
            var fileName = document.getElementById('file_name');

            ['dragenter', 'dragover', 'dragleave', 'drop'].forEach(eventName => {
                dropZone.addEventListener(eventName, preventDefaults, false);
            });

            function preventDefaults(e) {
                e.preventDefault();
                e.stopPropagation();
            }

            ['dragenter', 'dragover'].forEach(eventName => {
                dropZone.addEventListener(eventName, highlight, false);
            });

            ['dragleave', 'drop'].forEach(eventName => {
                dropZone.addEventListener(eventName, unhighlight, false);
            });

            function highlight(e) {
                dropZone.classList.add('highlight');
            }

            function unhighlight(e) {
                dropZone.classList.remove('highlight');
            }

            dropZone.addEventListener('drop', handleDrop, false);

            function handleDrop(e) {
                fileInput.files = e.dataTransfer.files;
                showFileName();
            }

            dropZone.onclick = function() {
                fileInput.click();
            }

            fileInput.addEventListener('change', showFileName, false);

            function showFileName() {
                var file = fileInput.files[0];
                if (file) {
                    dropZone.classList.add('file-dropped');
                    dropText.textContent = 'File selected:';
                    fileName.textContent = file.name;
                }
            }

            function uploadFile() {
                var file = fileInput.files[0];
                if (file) {
                    var reader = new FileReader();
                    reader.onload = function(e) {
                        var content = e.target.result;
                        google.colab.kernel.invokeFunction('notebook.uploadJSON', [content], {});
                    };
                    reader.readAsText(file);
                } else {
                    alert("Please select a file first.");
                }
            }
        </script>
    </body>
    </html>
    """

    display(HTML(html_content))

    # Add a back button
    back_button = widgets.Button(description='Back to Main')
    back_button.on_click(show_main_page)
    display(back_button)

def upload_json_to_firebase(content):
    try:
        data = json.loads(content)
        root_ref = db.reference('/')
        root_ref.delete()
        print("All data has been deleted from the database.")
        result = root_ref.child('Data').set(data)
        print("JSON data has been added to the Firebase database.")
    except json.JSONDecodeError:
        print("Invalid JSON file")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Function to create the search page
def create_search_page():
    custom_css = """
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #ffffff;
        }
        .container {
            max-width: 400px;
            margin: 0 auto;
            padding: 20px;
            background-color: #3c1361;
            border-radius: 10px;
            box-shadow: 0 0 10px rgba(0,0,0,0.3);
        }
        h1 {
            color: #e6e6fa;
            text-align: center;
            margin-bottom: 20px;
            font-size: 24px;
        }
        .widget-label {
            min-width: 90px;
            color: #ffffff;
            font-weight: bold;
            font-size: 12px;
        }
        .widget-text {
            width: 100% !important;
            margin-bottom: 10px;
            color: white;
        }
        .widget-text input {
            border: 1px solid #8a2be2;
            border-radius: 4px;
            padding: 5px;
            width: 100px;
            box-sizing: border-box;
            background-color: #4a1a82;
            color: #e6e6fa;
            font-size: 12px;
        }
        .widget-button button {
            background-color: #8a2be2;
            color: white;
            border: none;
            padding: 8px 15px;
            border-radius: 4px;
            width: 40%;
            cursor: pointer;
            transition: background-color 0.3s;
            font-size: 14px;
            margin: 10px auto;
            display: block;
        }
        .widget-button button:hover {
            background-color: #6a0dad;
        }
        .output_area {
            margin-top: 20px;
            padding: 10px;
            background-color: #4a1a82;
            border-radius: 4px;
            overflow-x: auto;
            max-height: 200px;
            overflow-y: auto;
            border: 1px solid #8a2be2;
            color: #e6e6fa;
        }
        table {
            width: 100%;
            border-collapse: collapse;
            margin-top: 10px;
            background-color: #4a1a82;
            font-size: 12px;
        }
        th, td {
            padding: 6px;
            text-align: left;
            border-bottom: 1px solid #6a0dad;
            color: #ffffff;
        }
        th {
            background-color: #6a0dad;
            color: white;
        }
        tr:nth-child(even) {
            background-color: #5c2a8a;
        }
    </style>
    """

    display(HTML(custom_css))

    # Create input widgets
    title = widgets.HTML(value="<h1>Search Activities</h1>")
    document_input = widgets.Text(description='Document:', style={'description_width': 'initial'})
    tab_input = widgets.Text(description='Tab:', style={'description_width': 'initial'})
    time_start_input = widgets.Text(description='Time Start:', style={'description_width': 'initial'})
    time_end_input = widgets.Text(description='Time End:', style={'description_width': 'initial'})
    user_input = widgets.Text(description='User:', style={'description_width': 'initial'})
    search_button = widgets.Button(description='Search', button_style='primary')
    output = widgets.Output()

    def search_firebase(document, tab, time_start, time_end, user):
        results = []
        for item in data:
            if isinstance(item, dict):
                time = item.get('Time')
                if (item.get('Document') == document or not document) and \
                   (item.get('Tab') == tab or not tab) and \
                   (not time_start or time >= time_start) and \
                   (not time_end or time <= time_end) and \
                   (item.get('User') == user or not user):
                    results.append(item)

        if results:
            table_html = "<table><tr><th>Time</th><th>User</th><th>Document</th><th>Tab</th><th>Description</th></tr>"
            for result in results:
                table_html += f"<tr><td>{result.get('Time', 'N/A')}</td><td>{result.get('User', 'N/A')}</td><td>{result.get('Document', 'N/A')}</td><td>{result.get('Tab', 'N/A')}</td><td>{result.get('Description', 'N/A')}</td></tr>"
            table_html += "</table>"
            return table_html
        else:
            return "<p>No matching data found</p>"

    def on_button_clicked(b):
        with output:
            clear_output()
            result = search_firebase(document_input.value, tab_input.value,
                                     time_start_input.value, time_end_input.value,
                                     user_input.value)
            display(HTML(result))

    search_button.on_click(on_button_clicked)

    # Create a vertical box layout
    vbox = widgets.VBox([
        title,
        document_input,
        tab_input,
        time_start_input,
        time_end_input,
        user_input,
        search_button,
        output
    ])

    # Add a back button
    back_button = widgets.Button(description='Back to Main')
    back_button.on_click(show_main_page)

    # Display widgets in a styled container
    display(HTML("<div class='container'>"))
    display(vbox)
    display(back_button)
    display(HTML("</div>"))

# Function to create the chart page
def create_chart_page():
    # Assuming 'data' is a list of dictionaries with 'User' field
    user_activity = {}
    for item in data:
        if isinstance(item, dict):
            user = item.get('User')
            if user:
                user_activity[user] = user_activity.get(user, 0) + 1

    sorted_names = sorted(user_activity.items(), key=lambda x: x[1], reverse=True)

    labels = [name for name, _ in sorted_names]
    sizes = [count for _, count in sorted_names]

    plt.figure(figsize=(10, 6))
    plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    plt.title('User Activity Distribution', fontsize=16)
    plt.axis('equal')

    # Save the plot to a bytes buffer
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_str = base64.b64encode(buf.getvalue()).decode('utf-8')

    # Close the matplotlib figure to free up memory
    plt.close()

    # Display the chart as an image in HTML
    html_content = f"""
    <html>
    <head>
        <title>User Activity Distribution</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                background-color: #f0f0f0;
                display: flex;
                flex-direction: column;
                align-items: center;
                padding: 20px;
            }}
            .container {{
                background-color: white;
                border-radius: 10px;
                padding: 20px;
                box-shadow: 0 0 10px rgba(0,0,0,0.1);
                max-width: 800px;
                width: 100%;
            }}
            h1 {{
                color: #333;
                text-align: center;
            }}
            img {{
                max-width: 100%;
                height: auto;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <h1>User Activity Distribution</h1>
            <img src="data:image/png;base64,{img_str}"/>
        </div>
    </body>
    </html>
    """

    display(HTML(html_content))

    # Add a back button
    back_button.on_click(show_main_page)
    display(back_button)

def create_most_frequent_activity_page():
    activity_counter = Counter()
    for item in data:
        if isinstance(item, dict):
            activity = item.get('Description')
            if activity:
                activity_counter[activity] += 1

    most_common = activity_counter.most_common(10)

    plt.figure(figsize=(12, 6))
    activities, counts = zip(*most_common)
    plt.bar(activities, counts)
    plt.title('Top 10 Most Frequent Activities', fontsize=16)
    plt.xlabel('Activity Description')
    plt.ylabel('Frequency')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_str = base64.b64encode(buf.getvalue()).decode('utf-8')
    plt.close()

    html_content = f"""
    <html>
    <head>
        <title>Most Frequent Activities</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                background-color: #f0f0f0;
                display: flex;
                flex-direction: column;
                align-items: center;
                padding: 20px;
            }}
            .container {{
                background-color: white;
                border-radius: 10px;
                padding: 20px;
                box-shadow: 0 0 10px rgba(0,0,0,0.1);
                max-width: 900px;
                width: 100%;
            }}
            h1 {{
                color: #333;
                text-align: center;
            }}
            img {{
                max-width: 100%;
                height: auto;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Top 10 Most Frequent Activities</h1>
            <img src="data:image/png;base64,{img_str}"/>
        </div>
    </body>
    </html>
    """

    display(HTML(html_content))

    back_button = widgets.Button(description='Back to Main')
    back_button.on_click(show_main_page)
    display(back_button)

def create_activity_distribution_over_time_page():
    activity_by_time = {}
    for item in data:
        if isinstance(item, dict):
            time = item.get('Time')
            if time:
                date = time.split()[0]  # Extract date part
                activity_by_time[date] = activity_by_time.get(date, 0) + 1

    dates = sorted(activity_by_time.keys())
    counts = [activity_by_time[date] for date in dates]

    plt.figure(figsize=(12, 6))
    plt.plot(dates, counts, marker='o')
    plt.title('Activity Distribution Over Time', fontsize=16)
    plt.xlabel('Date')
    plt.ylabel('Number of Activities')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_str = base64.b64encode(buf.getvalue()).decode('utf-8')
    plt.close()

    html_content = f"""
    <html>
    <head>
        <title>Activity Distribution Over Time</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                background-color: #f0f0f0;
                display: flex;
                flex-direction: column;
                align-items: center;
                padding: 20px;
            }}
            .container {{
                background-color: white;
                border-radius: 10px;
                padding: 20px;
                box-shadow: 0 0 10px rgba(0,0,0,0.1);
                max-width: 900px;
                width: 100%;
            }}
            h1 {{
                color: #333;
                text-align: center;
            }}
            img {{
                max-width: 100%;
                height: auto;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Activity Distribution Over Time</h1>
            <img src="data:image/png;base64,{img_str}"/>
        </div>
    </body>
    </html>
    """

    display(HTML(html_content))

    back_button = widgets.Button(description='Back to Main')
    back_button.on_click(show_main_page)
    display(back_button)

def create_on_most_frequent_worker_names_page():
    worker_counter = Counter()
    for item in data:
        if isinstance(item, dict):
            user = item.get('User')
            if user:
                worker_counter[user] += 1

    most_common = worker_counter.most_common(10)

    plt.figure(figsize=(12, 6))
    workers, counts = zip(*most_common)
    plt.bar(workers, counts)
    plt.title('Top 10 Most Active Workers', fontsize=16)
    plt.xlabel('Worker Name')
    plt.ylabel('Number of Activities')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_str = base64.b64encode(buf.getvalue()).decode('utf-8')
    plt.close()

    html_content = f"""
    <html>
    <head>
        <title>Most Active Workers</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                background-color: #f0f0f0;
                display: flex;
                flex-direction: column;
                align-items: center;
                padding: 20px;
            }}
            .container {{
                background-color: white;
                border-radius: 10px;
                padding: 20px;
                box-shadow: 0 0 10px rgba(0,0,0,0.1);
                max-width: 900px;
                width: 100%;
            }}
            h1 {{
                color: #333;
                text-align: center;
            }}
            img {{
                max-width: 100%;
                height: auto;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Top 10 Most Active Workers</h1>
            <img src="data:image/png;base64,{img_str}"/>
        </div>
    </body>
    </html>
    """

    display(HTML(html_content))

    back_button = widgets.Button(description='Back to Main')
    back_button.on_click(show_main_page)
    display(back_button)

def create_on_recent_activity_details_button_click_page():
    sorted_data = sorted(data, key=lambda x: x.get('Time', ''), reverse=True)
    recent_activities = sorted_data[:20]  # Get the 20 most recent activities

    html_content = """
    <html>
    <head>
        <title>Recent Activity Details</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f0f0f0;
                display: flex;
                flex-direction: column;
                align-items: center;
                padding: 20px;
            }
            .container {
                background-color: white;
                border-radius: 10px;
                padding: 20px;
                box-shadow: 0 0 10px rgba(0,0,0,0.1);
                max-width: 900px;
                width: 100%;
            }
            h1 {
                color: #333;
                text-align: center;
            }
            table {
                width: 100%;
                border-collapse: collapse;
                margin-top: 20px;
            }
            th, td {
                border: 1px solid #ddd;
                padding: 8px;
                text-align: left;
            }
            th {
                background-color: #f2f2f2;
                font-weight: bold;
            }
            tr:nth-child(even) {
                background-color: #f9f9f9;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Recent Activity Details</h1>
            <table>
                <tr>
                    <th>Time</th>
                    <th>User</th>
                    <th>Document</th>
                    <th>Tab</th>
                    <th>Description</th>
                </tr>
    """

    for activity in recent_activities:
        html_content += f"""
                <tr>
                    <td>{activity.get('Time', 'N/A')}</td>
                    <td>{activity.get('User', 'N/A')}</td>
                    <td>{activity.get('Document', 'N/A')}</td>
                    <td>{activity.get('Tab', 'N/A')}</td>
                    <td>{activity.get('Description', 'N/A')}</td>
                </tr>
        """

    html_content += """
            </table>
        </div>
    </body>
    </html>
    """

    display(HTML(html_content))

    back_button = widgets.Button(description='Back to Main')
    back_button.on_click(show_main_page)
    display(back_button)

def create_salary_page():
    custom_css = """
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #ffffff;
        }
        .container {
            max-width: 400px;
            margin: 0 auto;
            padding: 20px;
            background-color: #3c1361;
            border-radius: 10px;
            box-shadow: 0 0 10px rgba(0,0,0,0.3);
        }
        h1 {
            color: #e6e6fa;
            text-align: center;
            margin-bottom: 20px;
            font-size: 24px;
        }
        .widget-label {
            min-width: 90px;
            color: #ffffff;
            font-weight: bold;
            font-size: 12px;
        }
        .widget-text {
            width: 100% !important;
            margin-bottom: 10px;
            color: white;
        }
        .widget-text input {
            border: 1px solid #8a2be2;
            border-radius: 4px;
            padding: 5px;
            width: 100px;
            box-sizing: border-box;
            background-color: #4a1a82;
            color: #e6e6fa;
            font-size: 12px;
        }
        .widget-button button {
            background-color: #8a2be2;
            color: white;
            border: none;
            padding: 8px 15px;
            border-radius: 4px;
            width: 40%;
            cursor: pointer;
            transition: background-color 0.3s;
            font-size: 14px;
            margin: 10px auto;
            display: block;
        }
        .widget-button button:hover {
            background-color: #6a0dad;
        }
        .output_area {
            margin-top: 20px;
            padding: 10px;
            background-color: #4a1a82;
            border-radius: 4px;
            overflow-x: auto;
            max-height: 200px;
            overflow-y: auto;
            border: 1px solid #8a2be2;
            color: #e6e6fa;
        }
        table {
            width: 100%;
            border-collapse: collapse;
            margin-top: 10px;
            background-color: #4a1a82;
            font-size: 12px;
        }
        th, td {
            padding: 6px;
            text-align: left;
            border-bottom: 1px solid #6a0dad;
            color: #ffffff;
        }
        th {
            background-color: #6a0dad;
            color: white;
        }
        tr:nth-child(even) {
            background-color: #5c2a8a;
        }
    </style>
    """

    display(HTML(custom_css))

    title = widgets.HTML(value="<h1>Calculate Employee Salary</h1>")
    user_input = widgets.Text(description='User:', style={'description_width': 'initial'})
    rate_input = widgets.FloatText(description='Hourly Rate:', style={'description_width': 'initial'})
    calculate_button = widgets.Button(description='Calculate', button_style='primary')
    output = widgets.Output()

    def calculate_salary(user, rate):
        total_hours = 0
        for item in data:
            if isinstance(item, dict) and item.get('User') == user:
                timestamp = item.get('Time', '')
                if timestamp:
                    try:
                        time_obj = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
                        total_hours += 1
                    except ValueError:
                        continue

        salary = total_hours * rate
        return total_hours, salary

    def on_button_clicked(b):
        with output:
            clear_output()
            user = user_input.value
            rate = rate_input.value
            total_hours, salary = calculate_salary(user, rate)
            table_html = f"""
                <table>
                    <tr><th>User</th><th>Total Hours Worked</th><th>Total Salary</th></tr>
                    <tr><td>{user}</td><td>{total_hours}</td><td>${salary:.2f}</td></tr>
                </table>
            """
            display(HTML(table_html))

    calculate_button.on_click(on_button_clicked)

    vbox = widgets.VBox([
        title,
        user_input,
        rate_input,
        calculate_button,
        output
    ])

    back_button = widgets.Button(description='Back to Main')
    back_button.on_click(show_main_page)

    display(HTML("<div class='container'>"))
    display(vbox)
    display(back_button)
    display(HTML("</div>"))

# Function to show the main page
# Function to show the main page
def show_main_page(b=None):
    clear_output()
    main_page_html = """
    <html>
    <head>
        <style>
            body {
                font-family: Arial, sans-serif;
                background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
                margin: 0;
            }
            .container {
                background-color: rgba(255, 255, 255, 0.9);
                border-radius: 10px;
                padding: 30px;
                box-shadow: 0 10px 20px rgba(0,0,0,0.19), 0 6px 6px rgba(0,0,0,0.23);
                text-align: center;
            }
            h1 {
                color: #333;
                margin-bottom: 20px;
            }
            .button-container {
                display: flex;
                flex-direction: column;
                align-items: center;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Welcome, Admin!</h1>
            <div class="button-container">
                <!-- Buttons will be inserted here -->
            </div>
        </div>
    </body>
    </html>
    """
    display(HTML(main_page_html))
    display(widgets.VBox([upload_button, search_button, chart_button,
                          most_frequent_activity_button, activity_distribution_over_time_button,
                          most_frequent_worker_names_button, recent_activity_details_button, salary_button]))

# Button click handlers
def on_upload_button_click(b):
    clear_output()
    create_upload_page()

def on_search_button_click(b):
    clear_output()
    create_search_page()

def on_chart_button_click(b):
    clear_output()
    create_chart_page()

def on_most_frequent_activity_button_click(b):
    clear_output()
    create_most_frequent_activity_page()

def on_activity_distribution_over_time_button_click(b):
    clear_output()
    create_activity_distribution_over_time_page()

def on_most_frequent_worker_names_button_click(b):
    clear_output()
    create_on_most_frequent_worker_names_page()

def on_recent_activity_details_button_click(b):
    clear_output()
    create_on_recent_activity_details_button_click_page()

def on_salary_button_click(b):
    clear_output()
    create_salary_page()

# Create main page buttons
upload_button = widgets.Button(description="Go to Upload Page",
                               layout=widgets.Layout(width='300px', margin='10px 0'),
                               style=dict(button_color='#4CAF50'))
search_button = widgets.Button(description="Go to Search Page",
                               layout=widgets.Layout(width='300px', margin='10px 0'),
                               style=dict(button_color='#2196F3'))
chart_button = widgets.Button(description="Go to Chart Page",
                              layout=widgets.Layout(width='300px', margin='10px 0'),
                              style=dict(button_color='#FF9800'))
most_frequent_activity_button = widgets.Button(description="Most Frequent Activity",
                              layout=widgets.Layout(width='300px', margin='10px 0'),
                              style=dict(button_color='#E91E63'))
activity_distribution_over_time_button = widgets.Button(description="Activity Distribution Over Time",
                              layout=widgets.Layout(width='300px', margin='10px 0'),
                              style=dict(button_color='#9C27B0'))
most_frequent_worker_names_button = widgets.Button(description="Most Frequent Worker Names",
                              layout=widgets.Layout(width='300px', margin='10px 0'),
                              style=dict(button_color='#00BCD4'))
recent_activity_details_button = widgets.Button(description="Recent Activity Details",
                              layout=widgets.Layout(width='300px', margin='10px 0'),
                              style=dict(button_color='#FFC107'))
salary_button = widgets.Button(description="Employees Salary",
                              layout=widgets.Layout(width='300px', margin='10px 0'),
                              style=dict(button_color='#FF5733'))

# Attach click handlers
upload_button.on_click(on_upload_button_click)
search_button.on_click(on_search_button_click)
chart_button.on_click(on_chart_button_click)
most_frequent_activity_button.on_click(on_most_frequent_activity_button_click)
activity_distribution_over_time_button.on_click(on_activity_distribution_over_time_button_click)
most_frequent_worker_names_button.on_click(on_most_frequent_worker_names_button_click)
recent_activity_details_button.on_click(on_recent_activity_details_button_click)
salary_button.on_click(on_salary_button_click)

# Register the callback for file upload
from google.colab import output
output.register_callback('notebook.uploadJSON', upload_json_to_firebase)

# Display the main page initially
show_main_page()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/homework2-12bd2-firebase-adminsdk-d8w1y-65ac871075.json'